## <center> Packages & Connection </center>

In [2]:
import h3 as h3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats
from pyhive import presto
from keplergl import KeplerGl
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 300)

In [5]:
raw_list_hex_8 = pd.read_csv('/Users/rapido/local-datasets/affluence/processed/income_and _density_signal_list_hex_8.csv')
raw_fare_estimates_enriched = pd.read_csv('/Users/rapido/local-datasets/affluence/raw/raw_fare_estimates_enriched_10to16.csv')

In [6]:
df_fee_main_data = raw_fare_estimates_enriched.copy(deep=True)
print(df_fee_main_data.shape)
df_fee_main_data.head(2)

(15117222, 17)


,yyyymmdd,city,service_name,quarter_hour,time_bucket,pickup_cluster,pickup_hex_8,drop_cluster,drop_hex_8,customer_id,fare_estimate_id,surge_strategy,surge_percentage,dynamic_surge,dynamic_fare,sub_total,discount_amount
0,20230710,Bangalore,Auto,2015,Evening_Peak,Bellandur,8861892569fffff,Varthur,88618920ebfffff,5e591dcc1275a31e1e5833c9,64ac1af42a70c7ac0b2b56f8,surge_dashboard_rule,10.204082,0.00,10.0,108.0,0.0
1,20230710,Bangalore,Link,1800,Evening_Peak,Basavanagudi,8861892597fffff,Cambridge Layout Ulsoor,8861892ed1fffff,62c843ef0eef7f6e35f11140,64abfc344f76e21a1142b7dc,rain,39.517694,42.77,0.0,151.0,10.0


In [7]:
df_list_hex_8 = raw_list_hex_8.copy(deep=True)
print(df_list_hex_8.shape)
df_list_hex_8.head(2)

(2684, 3)


,pickup_hex_8,income_signal,density_signal
0,88618920a3fffff,High Income,High FE
1,8861892581fffff,High Income,High FE


##

In [8]:
df_fee_main_data.service_name.describe()

count     15117222
unique           2
top           Auto
freq       7575099
Name: service_name, dtype: object

In [9]:
df_fee = df_fee_main_data[df_fee_main_data['service_name'].isin(['Link'])]
df_fee.head(2)

,yyyymmdd,city,service_name,quarter_hour,time_bucket,pickup_cluster,pickup_hex_8,drop_cluster,drop_hex_8,customer_id,fare_estimate_id,surge_strategy,surge_percentage,dynamic_surge,dynamic_fare,sub_total,discount_amount
1,20230710,Bangalore,Link,1800,Evening_Peak,Basavanagudi,8861892597fffff,Cambridge Layout Ulsoor,8861892ed1fffff,62c843ef0eef7f6e35f11140,64abfc344f76e21a1142b7dc,rain,39.517694,42.77,0.0,151.0,10.0
4,20230710,Bangalore,Link,1015,Morning_Peak,Basaveshwar Nagar,886014586bfffff,Yeshwanthpur,886014595bfffff,6399734853245a499edb0326,64ab8e968d1303f14c3e0cf9,no_surge,0.000000,0.00,0.0,47.0,0.0


In [10]:
#df_fee['time_period'] = np.where(df_fee)

In [11]:
df_fee_with_signal = pd.merge(
                            df_fee, 
                            df_list_hex_8,
                            how = 'left',
                            left_on = ['pickup_hex_8'],
                            right_on = ['pickup_hex_8']
                            )
df_fee_with_signal.head(2)

,yyyymmdd,city,service_name,quarter_hour,time_bucket,pickup_cluster,pickup_hex_8,drop_cluster,drop_hex_8,customer_id,fare_estimate_id,surge_strategy,surge_percentage,dynamic_surge,dynamic_fare,sub_total,discount_amount,income_signal,density_signal
0,20230710,Bangalore,Link,1800,Evening_Peak,Basavanagudi,8861892597fffff,Cambridge Layout Ulsoor,8861892ed1fffff,62c843ef0eef7f6e35f11140,64abfc344f76e21a1142b7dc,rain,39.517694,42.77,0.0,151.0,10.0,High Income,High FE
1,20230710,Bangalore,Link,1015,Morning_Peak,Basaveshwar Nagar,886014586bfffff,Yeshwanthpur,886014595bfffff,6399734853245a499edb0326,64ab8e968d1303f14c3e0cf9,no_surge,0.000000,0.00,0.0,47.0,0.0,High Income,Low FE


In [12]:
df_fee_with_signal.surge_strategy.unique()

array(['rain', 'no_surge', 'hot_cold_surge', 'mismatch_generic',
       'mismatch_gradient', 'surge_dashboard_rule', 'circuit_broken'],
      dtype=object)

In [13]:
df_fee_with_signal_v1 = df_fee_with_signal[df_fee_with_signal['density_signal'].isin(['High FE']) 
                                           #&
                                           #df_fee_with_signal['surge_strategy'].isin(['surge_dashboard_rule'])
                                          ]

In [14]:
df_fee_with_signal_v1.groupby(['yyyymmdd', 'quarter_hour','income_signal','density_signal', 'surge_strategy']).agg({
    'fare_estimate_id' : 'count',
    'dynamic_surge' : 'sum',
    'dynamic_fare' : 'sum',
    'sub_total' : 'sum'
})

fare_estimate_id  \
yyyymmdd quarter_hour income_signal density_signal surge_strategy                     
20230710 0            High Income   High FE        no_surge                    3069   
                      Low Income    High FE        hot_cold_surge                 1   
                                                   no_surge                     478   
         15           High Income   High FE        no_surge                    2537   
                      Low Income    High FE        no_surge                     368   
...                                                                             ...   
20230716 2330         High Income   High FE        no_surge                    3740   
                      Low Income    High FE        no_surge                     453   
         2345         High Income   High FE        hot_cold_surge                 1   
                                                   no_surge                    3179   
                      Low Income    High FE        no_surge                     466   

                                                                   dynamic_surge  \
yyyymmdd quarter_hour income_signal density_signal surge_strategy                  
20230710 0            High Income   High FE        no_surge                 0.00   
                      Low Income    High FE        hot_cold_surge          70.03   
                                                   no_surge                 0.00   
         15           High Income   High FE        no_surge                 0.00   
                      Low Income    High FE        no_surge                 0.00   
...                                                                          ...   
20230716 2330         High Income   High FE        no_surge                 0.00   
                      Low Income    High FE        no_surge                 0.00   
         2345         High Income   High FE        hot_cold_surge          70.38   
                                                   no_surge                 0.00   
                      Low Income    High FE        no_surge                 0.00   

                                                                   dynamic_fare  \
yyyymmdd quarter_hour income_signal density_signal surge_strategy                 
20230710 0            High Income   High FE        no_surge                 0.0   
                      Low Income    High FE        hot_cold_surge           0.0   
                                                   no_surge                 0.0   
         15           High Income   High FE        no_surge                 0.0   
                      Low Income    High FE        no_surge                 0.0   
...                                                                         ...   
20230716 2330         High Income   High FE        no_surge                 0.0   
                      Low Income    High FE        no_surge                 0.0   
         2345         High Income   High FE        hot_cold_surge           0.0   
                                                   no_surge                 0.0   
                      Low Income    High FE        no_surge                 0.0   

                                                                   sub_total  
yyyymmdd quarter_hour income_signal density_signal surge_strategy             
20230710 0            High Income   High FE        no_surge         392246.0  
                      Low Income    High FE        hot_cold_surge      200.0  
                                                   no_surge          78891.0  
         15           High Income   High FE        no_surge         317457.0  
                      Low Income    High FE        no_surge          67366.0  
...                                                                      ...  
20230716 2330         High Income   High FE        no_surge         458176.0  
                      Low Income    High FE        

In [15]:
df_analysis1 = df_fee_with_signal_v1.groupby(['yyyymmdd','income_signal','density_signal', 'surge_strategy']).agg({
    'fare_estimate_id' : 'count',
    'dynamic_surge' : 'sum',
    'dynamic_fare' : 'sum',
    'sub_total' : 'sum'
}).reset_index()
df_analysis1

,yyyymmdd,income_signal,density_signal,surge_strategy,fare_estimate_id,dynamic_surge,dynamic_fare,sub_total
0,20230710,High Income,High FE,circuit_broken,11543,0.00,0.0,1341258.0
1,20230710,High Income,High FE,hot_cold_surge,94865,3033675.25,0.0,12987703.0
2,20230710,High Income,High FE,mismatch_generic,76187,925556.06,0.0,7171772.0
3,20230710,High Income,High FE,mismatch_gradient,62880,1786348.03,0.0,7064839.0
4,20230710,High Income,High FE,no_surge,400808,0.00,0.0,36301833.0
5,20230710,High Income,High FE,rain,433283,12793023.67,0.0,47741700.0
6,20230710,High Income,High FE,surge_dashboard_rule,33307,796570.53,0.0,2899596.0
7,20230710,Low Income,High FE,circuit_broken,1093,0.00,0.0,163383.0
8,20230710,Low Income,High FE,hot_cold_surge,7857,287047.53,0.0,1329962.0
9,20230710,Low Income,High FE,mismatch_generic,6802,104229.72,0.0,806143.0


In [16]:
df_analysis3 = df_fee_with_signal_v1.groupby(['yyyymmdd','income_signal', 
                                              'density_signal', 'surge_strategy', 'time_bucket']).agg({
    'fare_estimate_id' : 'count',
    'dynamic_surge' : 'sum',
    'dynamic_fare' : 'sum',
    'sub_total' : 'sum'
}).reset_index()
df_analysis3

,yyyymmdd,income_signal,density_signal,surge_strategy,time_bucket,fare_estimate_id,dynamic_surge,dynamic_fare,sub_total
0,20230710,High Income,High FE,circuit_broken,Afternoon,2250,0.00,0.0,316535.0
1,20230710,High Income,High FE,circuit_broken,Evening_Peak,4579,0.00,0.0,504596.0
2,20230710,High Income,High FE,circuit_broken,Morning_Peak,4240,0.00,0.0,456187.0
3,20230710,High Income,High FE,circuit_broken,Rest_Morning,474,0.00,0.0,63940.0
4,20230710,High Income,High FE,hot_cold_surge,Afternoon,13672,421757.01,0.0,2009837.0
...,...,...,...,...,...,...,...,...,...
354,20230716,Low Income,High FE,no_surge,Morning_Peak,7609,0.00,0.0,942407.0
355,20230716,Low Income,High FE,no_surge,Rest_Evening,4640,0.00,0.0,694376.0
356,20230716,Low Income,High FE,no_surge,Rest_Morning,7097,0.00,0.0,1118482.0
357,20230716,Low Income,High FE,rain,Afternoon,1191,32372.24,0.0,157412.0


In [18]:
df_analysis1['surge_percentage'] = round((df_analysis1['dynamic_surge']+df_analysis1['dynamic_fare'])*100/(df_analysis1['sub_total'] - (df_analysis1['dynamic_surge']+df_analysis1['dynamic_fare'])), 2)
#df_analysis2['surge_percentage'] = round((df_analysis2['dynamic_surge']+df_analysis2['dynamic_fare'])*100/(df_analysis2['sub_total'] - (df_analysis2['dynamic_surge']+df_analysis2['dynamic_fare'])), 2)
df_analysis3['surge_percentage'] = round((df_analysis3['dynamic_surge']+df_analysis3['dynamic_fare'])*100/(df_analysis3['sub_total'] - (df_analysis3['dynamic_surge']+df_analysis3['dynamic_fare'])), 2)

In [ ]:
#dynamic_surge+dynamic_fare/(sub_total-(dynamic_surge+dynamic_fare))

In [19]:
df_analysis_1 = df_analysis1.groupby(['yyyymmdd','income_signal','density_signal']).agg({'fare_estimate_id' : 'sum'}).reset_index()
df_analysis_1.rename(columns = {'fare_estimate_id' : 'total_fe'}, inplace = True)
df_analysis_1

,yyyymmdd,income_signal,density_signal,total_fe
0,20230710,High Income,High FE,1112873
1,20230710,Low Income,High FE,68585
2,20230711,High Income,High FE,940027
3,20230711,Low Income,High FE,53240
4,20230712,High Income,High FE,844178
5,20230712,Low Income,High FE,50054
6,20230713,High Income,High FE,999346
7,20230713,Low Income,High FE,54368
8,20230714,High Income,High FE,810831
9,20230714,Low Income,High FE,48787


In [20]:
df_analysis1 = pd.merge(df_analysis1, \
                        df_analysis_1, \
                        how = 'left', \
                        left_on = ['yyyymmdd', 'income_signal','density_signal'], \
                        right_on = ['yyyymmdd','income_signal', 'density_signal']
                        )
df_analysis1['surged_fe_distr'] = round(df_analysis1['fare_estimate_id']*100.00/df_analysis1['total_fe'], 2)

In [21]:
df_analysis1.head(5)

,yyyymmdd,income_signal,density_signal,surge_strategy,fare_estimate_id,dynamic_surge,dynamic_fare,sub_total,surge_percentage,total_fe,surged_fe_distr
0,20230710,High Income,High FE,circuit_broken,11543,0.00,0.0,1341258.0,0.00,1112873,1.04
1,20230710,High Income,High FE,hot_cold_surge,94865,3033675.25,0.0,12987703.0,30.48,1112873,8.52
2,20230710,High Income,High FE,mismatch_generic,76187,925556.06,0.0,7171772.0,14.82,1112873,6.85
3,20230710,High Income,High FE,mismatch_gradient,62880,1786348.03,0.0,7064839.0,33.84,1112873,5.65
4,20230710,High Income,High FE,no_surge,400808,0.00,0.0,36301833.0,0.00,1112873,36.02


In [22]:
df_analysis_3 = df_analysis3.groupby(['yyyymmdd','income_signal','density_signal', 'time_bucket']).agg({'fare_estimate_id' : 'sum'}).reset_index()
df_analysis_3.rename(columns = {'fare_estimate_id' : 'total_fe'}, inplace = True)
df_analysis_3

,yyyymmdd,income_signal,density_signal,time_bucket,total_fe
0,20230710,High Income,High FE,Afternoon,180796
1,20230710,High Income,High FE,Evening_Peak,555323
2,20230710,High Income,High FE,Morning_Peak,261542
3,20230710,High Income,High FE,Rest_Evening,31513
4,20230710,High Income,High FE,Rest_Morning,83699
5,20230710,Low Income,High FE,Afternoon,14146
6,20230710,Low Income,High FE,Evening_Peak,21747
7,20230710,Low Income,High FE,Morning_Peak,16852
8,20230710,Low Income,High FE,Rest_Evening,2889
9,20230710,Low Income,High FE,Rest_Morning,12951


In [23]:
df_analysis3 = pd.merge(df_analysis3, \
                        df_analysis_3, \
                        how = 'left', \
                        left_on = ['yyyymmdd','income_signal','density_signal', 'time_bucket'], \
                        right_on = ['yyyymmdd','income_signal','density_signal', 'time_bucket']
                        )
df_analysis3['surged_fe_distr'] = round(df_analysis3['fare_estimate_id']*100.00/df_analysis3['total_fe'], 2)

In [24]:
df_analysis3.head(5)

,yyyymmdd,income_signal,density_signal,surge_strategy,time_bucket,fare_estimate_id,dynamic_surge,dynamic_fare,sub_total,surge_percentage,total_fe,surged_fe_distr
0,20230710,High Income,High FE,circuit_broken,Afternoon,2250,0.00,0.0,316535.0,0.00,180796,1.24
1,20230710,High Income,High FE,circuit_broken,Evening_Peak,4579,0.00,0.0,504596.0,0.00,555323,0.82
2,20230710,High Income,High FE,circuit_broken,Morning_Peak,4240,0.00,0.0,456187.0,0.00,261542,1.62
3,20230710,High Income,High FE,circuit_broken,Rest_Morning,474,0.00,0.0,63940.0,0.00,83699,0.57
4,20230710,High Income,High FE,hot_cold_surge,Afternoon,13672,421757.01,0.0,2009837.0,26.56,180796,7.56


## surge_strategy 

In [25]:
pd.pivot_table(df_analysis1, \
               values= ['surge_percentage','surged_fe_distr'] , index=['income_signal', 'density_signal', 'surge_strategy'],
                       columns=['yyyymmdd'])

surge_percentage           \
yyyymmdd                                                  20230710 20230711   
income_signal density_signal surge_strategy                                   
High Income   High FE        circuit_broken                   0.00     0.00   
                             hot_cold_surge                  30.48    30.01   
                             mismatch_generic                14.82    14.83   
                             mismatch_gradient               33.84    33.57   
                             no_surge                         0.00     0.00   
                             rain                            36.61    29.63   
                             surge_dashboard_rule            37.88    37.87   
Low Income    High FE        circuit_broken                   0.00     0.00   
                             hot_cold_surge                  27.52    28.54   
                             mismatch_generic                14.85    14.84   
                             mismatch_gradient               33.96    24.41   
                             no_surge                         0.00     0.00   
                             rain                            36.73    29.68   
                             surge_dashboard_rule            35.45    34.13   

                                                                              \
yyyymmdd                                          20230712 20230713 20230714   
income_signal density_signal surge_strategy                                    
High Income   High FE        circuit_broken           0.00     0.00     0.00   
                             hot_cold_surge          28.08    26.69    26.47   
                             mismatch_generic        14.48    13.52    13.27   
                             mismatch_gradient       40.55    41.48    44.39   
                             no_surge                 0.00     0.00     0.00   
                             rain                    26.33    26.82    26.00   
                             surge_dashboard_rule    37.97    34.25      NaN   
Low Income    High FE        circuit_broken           0.00     0.00     0.00   
                             hot_cold_surge          25.86    25.25    26.05   
                             mismatch_generic        14.57    13.52    13.70   
                             mismatch_gradient       40.57    35.77    36.15   
                             no_surge                 0.00     0.00     0.00   
                             rain                    27.15    27.23    27.09   
                             surge_dashboard_rule    34.55    29.53      NaN   

                                                                     \
yyyymmdd                                          20230715 20230716   
income_signal density_signal surge_strategy                           
High Income   High FE        circuit_broken           0.00     0.00   
                             hot_cold_surge          19.99    22.57   
                             mismatch_generic        13.32    13.22   
                             mismatch_gradient       37.58    27.30   
                             no_surge                 0.00     0.00   
                             rain                    26.39    25.50   
                             surge_dashboard_rule      NaN      NaN   
Low Income    High FE        circuit_broken           0.00     0.00   
                             hot_cold_surge          20.22    24.22   
                             mismatch_generic        14.19    14.85   
                             mismatch_gradient       38.29    31.60   
                             no_surge                 0.00     0.00   
                             rain                    28.14    26.28   
                             surge_dashboard_rule      NaN      NaN   

                                                  surged_fe_distr           \
yyyymmdd                                                 20

## Peak view vs surge_strategy

In [26]:
pd.pivot_table(df_analysis3, values= ['surge_percentage', 'surged_fe_distr'] , index=['income_signal','time_bucket','surge_strategy'],
                       columns=['yyyymmdd'])

surge_percentage           \
yyyymmdd                                                20230710 20230711   
income_signal time_bucket  surge_strategy                                   
High Income   Afternoon    circuit_broken                   0.00     0.00   
                           hot_cold_surge                  26.56    26.33   
                           mismatch_generic                14.80    14.81   
                           mismatch_gradient               46.27    23.38   
                           no_surge                         0.00     0.00   
                           rain                            39.53    29.86   
                           surge_dashboard_rule              NaN      NaN   
              Evening_Peak circuit_broken                   0.00     0.00   
                           hot_cold_surge                  33.22    31.01   
                           mismatch_generic                14.81    14.82   
                           mismatch_gradient               35.26    33.57   
                           no_surge                         0.00     0.00   
                           rain                            36.18    29.63   
              Morning_Peak circuit_broken                   0.00     0.00   
                           hot_cold_surge                  29.26    32.38   
                           mismatch_generic                14.82    14.83   
                           mismatch_gradient               33.30    33.94   
                           no_surge                         0.00     0.00   
                           rain                            39.46      NaN   
                           surge_dashboard_rule            37.88    37.87   
              Rest_Evening circuit_broken                    NaN     0.00   
                           hot_cold_surge                  26.46    24.98   
                           mismatch_generic                13.61    15.40   
                           mismatch_gradient               20.02    23.56   
                           no_surge                         0.00     0.00   
                           rain                            29.61    27.83   
              Rest_Morning circuit_broken                   0.00     0.00   
                           hot_cold_surge                  33.22    26.50   
                           mismatch_generic                14.82    14.83   
                           mismatch_gradient               22.21    22.27   
                           no_surge                         0.00     0.00   
                           rain                              NaN      NaN   
Low Income    Afternoon    circuit_broken                   0.00     0.00   
                           hot_cold_surge                  24.47    28.63   
                           mismatch_generic                14.83    14.83   
                           mismatch_gradient               46.23    24.50   
                           no_surge                         0.00     0.00   
                           rain                            39.53      NaN   
              Evening_Peak circuit_broken                   0.00     0.00   
                           hot_cold_surge                  23.59    27.62   
                           mismatch_generic                14.80    14.82   
                           mismatch_gradient               32.92    23.99   
                           no_surge                         0.00     0.00   
                           rain                            35.85    29.68   
              Morning_Peak circuit_broken                   0.00     0.00   
                           hot_cold_surge                  26.29    27.63   
                           mismatch_generic                14.86    14.86   
                           mismatch_gradient               31.27    25.16   
                           no_surge                         0.00     0.00   
                           surge_dashboard_r

#

In [27]:
df_analysis1['surge_strategy'].unique()

array(['circuit_broken', 'hot_cold_surge', 'mismatch_generic',
       'mismatch_gradient', 'no_surge', 'rain', 'surge_dashboard_rule'],
      dtype=object)

In [28]:
pd.pivot_table(df_analysis1[df_analysis1['surge_strategy'].isin(['circuit_broken', 'hot_cold_surge', 'mismatch_generic',
       'mismatch_gradient', 'rain', 'surge_dashboard_rule']) ], \
               values= ['surge_percentage','surged_fe_distr'], \
               index=['income_signal', 'density_signal', 'surge_strategy'], \
               columns=['yyyymmdd']
              ).describe()

surge_percentage                                              \
yyyymmdd         20230710   20230711   20230712   20230713   20230714   
count           12.000000  12.000000  12.000000  12.000000  10.000000   
mean            25.178333  23.125833  24.175833  22.838333  21.312000   
std             14.137517  12.856771  14.211303  13.356571  14.484457   
min              0.000000   0.000000   0.000000   0.000000   0.000000   
25%             14.842500  14.837500  14.547500  13.520000  13.377500   
50%             32.160000  29.085000  26.740000  26.755000  26.025000   
75%             35.740000  30.900000  35.405000  30.710000  26.935000   
max             37.880000  37.870000  40.570000  41.480000  44.390000   

                               surged_fe_distr                        \
yyyymmdd   20230715   20230716        20230710   20230711   20230712   
count     10.000000  10.000000       12.000000  12.000000  12.000000   
mean      19.812000  18.554000       10.075833   6.970000   6.259167   
std       13.452574  11.222663       11.712505   5.367869   3.622655   
min        0.000000   0.000000        1.040000   1.030000   1.290000   
25%       13.537500  13.627500        2.695000   3.600000   3.882500   
50%       20.105000  23.395000        6.250000   6.450000   5.825000   
75%       27.702500  26.085000       10.305000   7.977500   8.455000   
max       38.290000  31.600000       38.930000  20.940000  13.770000   

                                                    
yyyymmdd   20230713  20230714   20230715  20230716  
count     12.000000  10.00000  10.000000  10.00000  
mean       8.384167   5.48600   8.104000   5.34900  
std       11.756294   3.46972   9.590803   6.84258  
min        1.010000   0.73000   0.240000   0.32000  
25%        2.587500   2.42500   1.242500   1.07750  
50%        3.530000   5.84000   4.430000   2.85500  
75%        5.717500   8.38250   9.467500   4.29750  
max       36.140000  10.56000  26.720000  18.32000

In [29]:
pd.pivot_table(df_analysis3[df_analysis3['surge_strategy'] \
                            .isin(['circuit_broken', 'hot_cold_surge', 'mismatch_generic', \
                                   'mismatch_gradient', 'rain', 'surge_dashboard_rule']) ], \
               values= ['surge_percentage','surged_fe_distr'] , \
               index=['income_signal','surge_strategy'], \
               columns=['time_bucket']
              ).describe()

surge_percentage                                         \
time_bucket        Afternoon Evening_Peak Morning_Peak Rest_Evening   
count              11.000000    10.000000    11.000000    10.000000   
mean               20.907870    20.821714    24.189535    17.880590   
std                12.570093    13.110422    14.732839    10.674627   
min                 0.000000     0.000000     0.000000     0.000000   
25%                13.708333    14.242143    14.055000    13.979583   
50%                23.832857    25.951429    28.967143    22.309048   
75%                28.592333    28.879643    35.438750    25.249643   
max                37.111667    37.285714    39.460000    28.805000   

                         surged_fe_distr                            \
time_bucket Rest_Morning       Afternoon Evening_Peak Morning_Peak   
count           9.000000       11.000000    10.000000    11.000000   
mean           16.440794        6.734636    12.137857     7.707922   
std            10.598560        6.294671    12.344011     4.589936   
min             0.000000        0.000000     2.295714     0.220000   
25%            14.187500        2.056071     2.797500     4.831429   
50%            20.000000        4.620000     5.665714     8.395000   
75%            21.596667       11.878333    17.060357    10.037262   
max            30.434286       19.716000    34.104286    13.940000   

                                       
time_bucket Rest_Evening Rest_Morning  
count          10.000000     9.000000  
mean            0.120490     2.118360  
std             0.091083     2.014930  
min             0.028000     0.000000  
25%             0.047917     0.598571  
50%             0.075000     1.280000  
75%             0.216607     3.944286  
max             0.251429     5.720000

##

Mean value of q1, q2, q3 from day and peak-wise 

- Decrease surge percentage  Q1 -> ~14%
- BAU surge percentage Q2 -> ~26%
- Increase surge percentage Q3 -> ~31%

In [139]:
## Check weekly surge % : important 